In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

dataset_dir = "archive_2/train_test"

def preprocess_image(image_path, target_size=(384, 384)):
    with Image.open(image_path) as image:
        image = image.convert("RGB")
    image = image.resize(target_size)
    preprocessed_image = np.array(image) / 255.0
    return preprocessed_image

def is_authentic(filename):
    prefix = filename[:2]
    return (prefix == "au" or prefix == "Au")

images = []
labels = []

for filename in os.listdir(dataset_dir):
    if filename.startswith(".DS_Store"):
        continue
    file_path = os.path.join(dataset_dir, filename)
    preprocessed_image = preprocess_image(file_path)
    images.append(preprocessed_image)
    label = is_authentic(filename)
    labels.append(label)

images = np.array(images)
labels = np.array(labels)

data_augmentation = ImageDataGenerator(
    rotation_range = 25,
    height_shift_range = 0.25,
    width_shift_range = 0.25,
    vertical_flip = True,
    horizontal_flip = True,
    zoom_range = 0.25
)

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define the model architecture
base = tf.keras.applications.EfficientNetV2S(include_top=False, input_shape=train_images.shape[1:])

x = tf.keras.layers.GlobalAveragePooling2D()(base.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=base.input, outputs=output)

# Compile and train the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy', metrics=['accuracy'])

results = model.fit(data_augmentation.flow(train_images, train_labels, batch_size=32), epochs=20,
                    steps_per_epoch=len(train_images) // 32,
                    validation_data=(test_images, test_labels))

Metal device set to: Apple M1 Max
Epoch 1/20


2023-04-13 18:14:09.313175: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


190/190 [==============================] - 276s 1s/step - loss: 0.5128 - accuracy: 0.7355 - val_loss: 0.3857 - val_accuracy: 0.8263
Epoch 2/20
190/190 [==============================] - 266s 1s/step - loss: 0.3821 - accuracy: 0.8215 - val_loss: 0.5051 - val_accuracy: 0.7586
Epoch 3/20
190/190 [==============================] - 258s 1s/step - loss: 0.3253 - accuracy: 0.8475 - val_loss: 0.3161 - val_accuracy: 0.8441
Epoch 4/20
190/190 [==============================] - 257s 1s/step - loss: 0.2808 - accuracy: 0.8681 - val_loss: 0.3146 - val_accuracy: 0.8513
Epoch 5/20
190/190 [==============================] - 254s 1s/step - loss: 0.2644 - accuracy: 0.8738 - val_loss: 0.3185 - val_accuracy: 0.8349
Epoch 6/20
190/190 [==============================] - 254s 1s/step - loss: 0.2231 - accuracy: 0.8913 - val_loss: 0.2846 - val_accuracy: 0.8533
Epoch 7/20
190/190 [==============================] - 253s 1s/step - loss: 0.2147 - accuracy: 0.9015 - val_loss: 0.2890 - val_accuracy: 0.8546
Epoch 8/20

In [2]:
# Convert and save the model as a TensorFlow Lite file
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('manipulation_detector_config_8.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/x9/lnqw3ctd65g1t8sjc2w_k5hm0000gn/T/tmpoad7ppby/assets


INFO:tensorflow:Assets written to: /var/folders/x9/lnqw3ctd65g1t8sjc2w_k5hm0000gn/T/tmpoad7ppby/assets
2023-04-13 19:42:32.755161: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-04-13 19:42:32.755578: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.


In [3]:
# Evaluate the model on the testing set and print the accuracy
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Test accuracy:", test_acc)

48/48 [==============================] - 10s 201ms/step - loss: 0.3272 - accuracy: 0.8914
Test accuracy: 0.8914473652839661
